In [1]:
%run solution.ipynb

In [2]:
def calculate_delta_r9(T):
    return G0('AlCl3', T) + G0('NH3', T) - G0('AlN', T) - 3 * G0('HCl', T)

def calculate_delta_r10(T):
    return G0('GaCl', T) + G0('NH3', T) - G0('GaN', T) - G0('HCl', T) - G0('H2', T)

In [3]:
Pa = 100000

def K9(T, R = 8.314):
    return math.exp(-1 * calculate_delta_r9(T) / (R * T)) / Pa

def K10(T, R = 8.314):
    return math.exp(-1 * calculate_delta_r10(T) / (R * T))

In [4]:
T = 1100.0
print(K9(T))
print(K10(T))

3.100281771447924e-08
0.13120264119198397


In [5]:
print(D('NH3', T))
print(D('GaCl', T))
print(D('AlCl3', T))
print(D('HCl', T))
print(D('H2', T))

0.0007886217210652681
0.0004891294645967424
0.00039166855641119605
0.0007030166473273947
0.0022512831014624562


In [6]:

def run(k):
    PG = {
       'NH3':1500,
       'GaCl':(30 * k),
       'AlCl3':(30 * (1 - k)),
       'H2':0,
       'HCl':0,
       'N2':98470
    }


    x, AlCl3, GaCl, NH3, HCl, H2 = sy.symbols('x AlCl3 GaCl NH3 HCl H2')
    Xmat = sy.Matrix([
        AlCl3 * NH3 - K9(T) * x * (HCl)**3,
        GaCl * NH3 - K10(T) * (1 - x) * HCl * H2,
        D('HCl', T) * (PG['HCl'] - HCl) + 2 * D('H2', T) * (PG['H2'] - H2) + 3 * D('NH3', T) * (PG['NH3'] - NH3),
        3 * D('AlCl3', T) * (PG['AlCl3'] - AlCl3) + D('GaCl', T) * (PG['GaCl'] - GaCl) + D('HCl', T) * (PG['HCl'] - HCl),
        D('AlCl3', T) * (PG['AlCl3'] - AlCl3) + D('GaCl', T) * (PG['GaCl'] - GaCl) - D('NH3', T) * (PG['NH3'] - NH3),
        (1 - x) * D('AlCl3', T) * (PG['AlCl3'] - AlCl3) - x * D('GaCl', T) * (PG['GaCl'] - GaCl),
        
    ])
    
    Y = [x, AlCl3, GaCl, NH3, HCl, H2]
    print(Xmat.jacobian(Y))
    ans = newton_method(Xmat, Y, [1, 1, 1, 1, 1, 1])
    print(ans)

for d in [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]:
    run(d)

Matrix([[-3.10028177144792e-8*HCl**3, NH3, 0, AlCl3, -9.30084531434377e-8*HCl**2*x, 0], [0.131202641191984*H2*HCl, 0, NH3, GaCl, -H2*(0.131202641191984 - 0.131202641191984*x), -HCl*(0.131202641191984 - 0.131202641191984*x)], [0, 0, 0, -0.00236586516319580, -0.000703016647327395, -0.00450256620292491], [0, -0.00117500566923359, -0.000489129464596742, 0, -0.000703016647327395, 0], [0, -0.000391668556411196, -0.000489129464596742, 0.000788621721065268, 0, 0], [0.000391668556411196*AlCl3 + 0.000489129464596742*GaCl - 0.0117500566923359, 0.000391668556411196*x - 0.000391668556411196, 0.000489129464596742*x, 0, 0, 0]])
12
[1.00000000000000, 2.63167361243703e-6, -8.07793566946316e-28, 1485.10051733584, 50.1412976759286, 1.94565518684175e-14]
Matrix([[-3.10028177144792e-8*HCl**3, NH3, 0, AlCl3, -9.30084531434377e-8*HCl**2*x, 0], [0.131202641191984*H2*HCl, 0, NH3, GaCl, -H2*(0.131202641191984 - 0.131202641191984*x), -HCl*(0.131202641191984 - 0.131202641191984*x)], [0, 0, 0, -0.00236586516319580